In [ ]:
#The first cell is modified version of Lab-1, the only thing we added is "targets" array to keep label of samples MALWARE=1 BENIGN=0
import os
from os import listdir
from os.path import isfile, join
import numpy as np
paths={'data/malware':1,'data/benign':0} #Data Path
def is_hex(s): #Creturn True if input string is Hexadecimal: Based on Opcode file structure,this function is required to validate lines that include OpCode
    try:
        int(s, 16)
        return True
    except ValueError:
        return False
print('Start to extract opcodes.....')  
total_files_opcode=[] #this array includes counted information for all samples 
total_opcode_dictionary=[] #this dictionay includes a unique set of opcodes found in all samples.
targets=[]
for path in list(paths.keys()): #Loops over Malware and Benign folders
    files=os.listdir(path)
    for file in files: #Loops overfiles in each folder
        targets.append(paths[path])
        if file.endswith('.opcode'): # Read File line by line to process each eine
            file_dic={} # The dictionary that contains files opcode and their count
            file_handler=open(os.path.join(path,file),'r')
            line=file_handler.readline() #Read First Line
            while line:
                if  len(line)>30 and line[8]==':' and is_hex(line[0:8]): #a strange line of code but there is no simple way to validate line includes OpCode!! (Better Soloution -> Regular Expresssion)
                    rest_of_line=line[32:].split(' ') 
                    opcode=rest_of_line[0]
                    if opcode in file_dic.keys():
                        file_dic[opcode]=file_dic[opcode]+1 #increament the count of opcode
                    else:
                        file_dic[opcode]=1 #first occurance of opcode
                    if opcode not in total_opcode_dictionary:
                        total_opcode_dictionary.append(opcode)
                line=file_handler.readline() #Read Newline
            total_files_opcode.append(file_dic) #sample added to main array of samples

print('Opcodes were extracted ') 
#Generate the final list 
final_list=np.zeros((len(total_files_opcode),len(total_opcode_dictionary)),dtype=int) # The size of array is num_of_samples(rows)*num_of_unique_opcodes(Columns)
for i in range(len(total_files_opcode)):
    for j in range(len(total_opcode_dictionary)):
        opcode=total_opcode_dictionary[j]
        if opcode in total_files_opcode[i].keys():
            final_list[i][j]=total_files_opcode[i][opcode] #The sample includes opcode
        else:
            final_list[i][j]=0 #The sample does not include opcode
print('Final list generated ')
print("Size of List: ",np.shape(final_list),'Size of targets:', np.shape(targets))
print('final_list and targets are ready to use for training a machine learning model')

In [ ]:
#Write Some Code here to train a Decision Tree Classifier with Malware Data
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

training_samples, testing_samples, training_targets, testing_targets = train_test_split(final_list, targets, test_size=0.4, random_state=0)
clf=DecisionTreeClassifier()
clf.fit(training_samples, training_targets)
predictions = clf.predict(testing_samples)
accuracy = 100.0 * accuracy_score(testing_targets, predictions)
print ("Decision Tree accuracy: " + str(accuracy))